In [ ]:
import warnings
warnings.filterwarnings('ignore')
from pyspark.sql.functions import col

from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("MyGoitSparkSandbox").getOrCreate()

## 1. Завантажте та прочитайте кожен CSV-файл як окремий DataFrame.

In [3]:
# Завантажуємо датасет
products_df = spark.read.csv('./data/products.csv', header=True)

# Виводимо перші 10 записів
products_df.show(10)

+----------+------------+-----------+-----+
|product_id|product_name|   category|price|
+----------+------------+-----------+-----+
|         1|   Product_1|     Beauty|  8.3|
|         2|   Product_2|       Home|  8.3|
|         3|   Product_3|Electronics|  9.2|
|         4|   Product_4|Electronics|  2.6|
|         5|   Product_5|Electronics|  9.4|
|         6|   Product_6|     Sports|  8.7|
|         7|   Product_7|     Beauty|  8.2|
|         8|   Product_8|     Sports|  1.0|
|         9|   Product_9|     Beauty|  6.0|
|        10|  Product_10|     Sports|  5.4|
+----------+------------+-----------+-----+
only showing top 10 rows



In [4]:
# Завантажуємо датасет
purchases_df = spark.read.csv('./data/purchases.csv', header=True)

# Виводимо перші 10 записів
purchases_df.show(10)

+-----------+-------+----------+----------+--------+
|purchase_id|user_id|product_id|      date|quantity|
+-----------+-------+----------+----------+--------+
|          1|     52|         9|2022-01-01|       1|
|          2|     93|        37|2022-01-02|       8|
|          3|     15|        33|2022-01-03|       1|
|          4|     72|        42|2022-01-04|       9|
|          5|     61|        44|2022-01-05|       6|
|          6|     21|        24|2022-01-06|       7|
|          7|     83|        15|2022-01-07|       7|
|          8|     87|        32|2022-01-08|       3|
|          9|     75|        32|2022-01-09|       2|
|         10|     75|        24|2022-01-10|       9|
+-----------+-------+----------+----------+--------+
only showing top 10 rows



In [5]:
# Завантажуємо датасет
users_df = spark.read.csv('./data/users.csv', header=True)

# Виводимо перші 10 записів
users_df.show(10)

+-------+-------+---+------------------+
|user_id|   name|age|             email|
+-------+-------+---+------------------+
|      1| User_1| 45| user1@example.com|
|      2| User_2| 48| user2@example.com|
|      3| User_3| 36| user3@example.com|
|      4| User_4| 46| user4@example.com|
|      5| User_5| 29| user5@example.com|
|      6| User_6| 39| user6@example.com|
|      7| User_7| 24| user7@example.com|
|      8| User_8| 44| user8@example.com|
|      9| User_9| 27| user9@example.com|
|     10|User_10| 43|user10@example.com|
+-------+-------+---+------------------+
only showing top 10 rows



## 2. Очистіть дані, видаляючи будь-які рядки з пропущеними значеннями.


In [6]:
# Видаляємо рядки з пропущеними значеннями
products_df = products_df.dropna()
purchases_df = purchases_df.dropna()
users_df = users_df.dropna()

# Виводимо перші 10 записів після очищення
products_df.show(10)
purchases_df.show(10)
users_df.show(10)

+----------+------------+-----------+-----+
|product_id|product_name|   category|price|
+----------+------------+-----------+-----+
|         1|   Product_1|     Beauty|  8.3|
|         2|   Product_2|       Home|  8.3|
|         3|   Product_3|Electronics|  9.2|
|         4|   Product_4|Electronics|  2.6|
|         5|   Product_5|Electronics|  9.4|
|         6|   Product_6|     Sports|  8.7|
|         7|   Product_7|     Beauty|  8.2|
|         8|   Product_8|     Sports|  1.0|
|         9|   Product_9|     Beauty|  6.0|
|        10|  Product_10|     Sports|  5.4|
+----------+------------+-----------+-----+
only showing top 10 rows

+-----------+-------+----------+----------+--------+
|purchase_id|user_id|product_id|      date|quantity|
+-----------+-------+----------+----------+--------+
|          1|     52|         9|2022-01-01|       1|
|          2|     93|        37|2022-01-02|       8|
|          3|     15|        33|2022-01-03|       1|
|          4|     72|        42|2022-01-

## 3. Визначте загальну суму покупок за кожною категорією продуктів.

In [7]:
joined_df = purchases_df.join(products_df, purchases_df.product_id == products_df.product_id)

grouped3 = joined_df.select('category', 'price', 'quantity').withColumn("purchases_sum", col("price") * (col("quantity")))

grouped3.groupBy('category').sum("purchases_sum").show()


+-----------+------------------+
|   category|sum(purchases_sum)|
+-----------+------------------+
|       Home|1523.4999999999998|
|     Sports|1802.4999999999998|
|Electronics|1174.7999999999997|
|   Clothing|             790.3|
|     Beauty| 459.8999999999999|
+-----------+------------------+



## 4. Визначте суму покупок за кожною категорією продуктів для вікової категорії від 18 до 25 включно.

In [8]:
joined_df = purchases_df.join(products_df, purchases_df.product_id == products_df.product_id).join(users_df, purchases_df.user_id == users_df.user_id)
joined_df = joined_df.where((col('age') >= 18) & (col('age') <= 25))
grouped4 = joined_df.select('category', 'price', 'quantity').withColumn("purchases_sum", col("price") * (col("quantity")))

grouped4.groupBy('category').sum("purchases_sum").show()

+-----------+------------------+
|   category|sum(purchases_sum)|
+-----------+------------------+
|       Home|             361.1|
|     Sports|310.49999999999994|
|Electronics|             249.6|
|   Clothing|             245.0|
|     Beauty|41.400000000000006|
+-----------+------------------+



## 5. Визначте частку покупок за кожною категорією товарів від сумарних витрат для вікової категорії від 18 до 25 років.

In [9]:
from pyspark.sql.functions import col, sum as _sum, round

joined_df = purchases_df.join(products_df, purchases_df.product_id == products_df.product_id).join(users_df, purchases_df.user_id == users_df.user_id)

filtered_df = joined_df.where((col('age') >= 18) & (col('age') <= 25))

category_totals_df = filtered_df.withColumn("purchase_amount", col("price") * col("quantity")).groupBy('category').agg(_sum("purchase_amount").alias("total_amount"))

total_amount = category_totals_df.agg(_sum("total_amount").alias("overall_total")).collect()[0]["overall_total"]

category_share_df = category_totals_df.withColumn("share", round(col("total_amount") / total_amount, 2))

category_share_df.select(['category', 'share']).show()


+-----------+-----+
|   category|share|
+-----------+-----+
|       Home|  0.3|
|     Sports| 0.26|
|Electronics| 0.21|
|   Clothing|  0.2|
|     Beauty| 0.03|
+-----------+-----+



## 6. Виберіть 3 категорії продуктів з найвищим відсотком витрат споживачами віком від 18 до 25 років.

In [10]:
category_share_df.select(['category', 'share']).orderBy(col('share').desc()).limit(3).show()

+-----------+-----+
|   category|share|
+-----------+-----+
|       Home|  0.3|
|     Sports| 0.26|
|Electronics| 0.21|
+-----------+-----+

